# START
The first cell below must always be run, regardless of if the pickle has already been created or not.

In [ ]:
import pandas as pd
from bio_embeddings.embed import # embedder of choice
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from tqdm import tqdm
from sklearn.decomposition import PCA
import numpy as np

# STOP
Only run the cells below (until next markdown cell) if the sequences are being embedded with the embedder of choice. First make sure the 'embedder of choice' comment has been ammended before running, in addition to changing the file names for reading the excel file and subsequently saving the embedded data into a pickle. <br>
Scroll down to 'Resume' if you are intending to open a pickle.

In [ ]:
# Opening the requisite excel file into pandas dataframe. First two entries of the df are printed to check data is correct
df = pd.read_excel('File name.xlsx', engine='openpyxl')
df.reset_index(inplace=True, drop=True)
df.head(2)

In [ ]:
# Initially making a copy of the dataframe, then dropping entries which do not contain a value in the enantiomer column
# The shape is then checked to see how many entries remain
df = df.copy()
df.dropna(subset=['Enantiomer'], inplace=True)
df.shape

In [ ]:
# Using bioembeddings to embed the protein sequences, and saving the output data into a pickle file as it is a lengthy
# process to run the code (25 mins). The pickle file can be opened (vide infra) in the future to access the embeddings.
# DO NOT RUN
embedder = # embedder of choice()
df['embedding'] = df['Sequence'].progress_apply(embedder.embed)
df['em_per_protein'] = df['embedding'].progress_apply(embedder.reduce_per_protein)

df.to_pickle('File name.pkl')
df.head(2)

# RESUME
In all instances where random_state is an argument, ensure the **same integer** is maintained for reproducibility purposes. <br> N.b. integer = 25

In [ ]:
SEED=25

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import f1_score, log_loss, roc_curve, auc, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize': (10, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')
sns.set_theme()
%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_pickle('File name.pkl')
df.head(2)

In [ ]:
X = list(df['em_per_protein'])
y = df['enantiomer binary']

In [ ]:
# Splitting the data into training and test sets, remember the random_state argument!
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=SEED)

## PCA analysis
Quick check for correlation between R and S selective sequences

In [ ]:
pca = PCA(n_components=2)
crds = pca.fit_transform(list(df['em_per_protein']))

pca_df = pd.DataFrame(crds, columns=['x', 'y'])

df['x'] = pca_df['x']
df['y'] = pca_df['y']
ax = sns.scatterplot(data=df, x='x', y='y', hue='Enantiomer', style='Enantiomer', s=100)

## k-Nearest Neighbours (k-NN)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)

knn_accuracy = knn.score(X_test, y_test)
knn_f1 = f1_score(y_test, y_pred_knn)
knn_logloss = log_loss(y_test, y_pred_knn)

print('The knn accuracy is {:.3f}'.format(knn_accuracy));
print('The knn f1 score is {:.3f}'.format(knn_f1));
print('The knn log loss is {:.3f}'.format(knn_logloss))

In [ ]:
y_scores_knn = knn.predict_proba(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_scores_knn[:,1])

roc_auc = auc(fpr, tpr)

plt.plot([0,1], [0,1], 'r--')
plt.plot(fpr, tpr, label='AUC = %0.2f' %roc_auc)
plt.legend(loc='lower right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('k-NN ROC curve')
plt.show();

In [ ]:
knn_cv_scores = cross_val_score(knn, X, y, cv=10, scoring='f1')
knn_cv_mean = knn_cv_scores.mean()
knn_cv_std = knn_cv_scores.std()
print('The k-NN cross-validation mean is {:.3f}'.format(knn_cv_mean));
print('The k-NN cross-validation standard deviation is {:.3f}'.format(knn_cv_std))

# Linear Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

logreg_accuracy = knn.score(X_test, y_test)
logreg_f1 = f1_score(y_test, y_pred_logreg)
logreg_logloss = log_loss(y_test, y_pred_logreg)

print('The Logistic Regression accuracy is {:.3f}'.format(logreg_accuracy));
print('The Logistic Regression f1 score is {:.3f}'.format(logreg_f1));
print('The Logistic Regression log loss is {:.3f}'.format(logreg_logloss))

In [ ]:
y_scores_logreg = logreg.predict_proba(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_scores_logreg[:,1])

roc_auc = auc(fpr, tpr)

plt.plot([0,1], [0,1], 'r--')
plt.plot(fpr, tpr, label='AUC = %0.2f' %roc_auc)
plt.legend(loc='lower right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC curve')
plt.show();

In [ ]:
logreg_cv_scores = cross_val_score(logreg, X, y, cv=10, scoring='f1')
logreg_cv_mean = logreg_cv_scores.mean()
logreg_cv_std = logreg_cv_scores.std()
print('The Logistic Regression cross-validation mean is {:.3f}'.format(logreg_cv_mean));
print('The Logistic Regression cross-validation standard deviation is {:.3f}'.format(logreg_cv_std))

# Voting Classifier Ensemble

In [ ]:
lr = LogisticRegression(random_state=SEED)
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier(random_state=SEED)
classifiers = [('Logistic Regression', lr),('K Nearest Neighbours', knn),('Classification Tree', dt)]

## Soft voting

In [ ]:
vc_soft = VotingClassifier(estimators=classifiers, voting='soft')
vc_soft.fit(X_train, y_train)
y_pred_soft = vc_soft.predict(X_test)

vc_soft_accuracy = vc_soft.score(X_test, y_test)
vc_soft_f1 = f1_score(y_test, y_pred_soft)
vc_soft_logloss = log_loss(y_test, y_pred_soft)

print('The Voting Classifier (soft) accuracy is {:.3f}'.format(vc_soft_accuracy));
print('The Voting Classifier (soft) f1 score is {:.3f}'.format(vc_soft_f1));
print('The Voting Classifier (soft) log loss is {:.3f}'.format(vc_soft_logloss))

In [ ]:
y_scores_soft = vc_soft.predict_proba(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_scores_soft[:,1])
roc_auc = auc(fpr, tpr)

y_pred_prob = vc_soft.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label='AUC = %0.2f' %roc_auc)
plt.legend(loc='lower right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Voting Classifier ensemble ROC curve')
plt.show();

In [ ]:
soft_cv_scores = cross_val_score(vc_soft, X, y, cv=10, scoring='f1')
soft_cv_mean = soft_cv_scores.mean()
soft_cv_std = soft_cv_scores.std()
print('The Voting Classifier (soft) cross-validation mean is {:.3f}'.format(soft_cv_mean));
print('The Voting Classifier (soft) cross-validation standard deviation is {:.3f}'.format(soft_cv_std))

## Hard voting

In [ ]:
vc_hard = VotingClassifier(estimators=classifiers, voting='hard')
vc_hard.fit(X_train, y_train)
y_pred_hard = vc_hard.predict(X_test)

vc_hard_accuracy = vc_soft.score(X_test, y_test)
vc_hard_f1 = f1_score(y_test, y_pred_hard)
vc_hard_logloss = log_loss(y_test, y_pred_hard)

print('The Voting Classifier (hard) accuracy is {:.3f}'.format(vc_hard_accuracy));
print('The Voting Classifier (hard) f1 score is {:.3f}'.format(vc_hard_f1));
print('The Voting Classifier (hard) log loss is {:.3f}'.format(vc_hard_logloss))

In [ ]:
hard_cv_scores = cross_val_score(vc_hard, X, y, cv=10, scoring='f1')
hard_cv_mean = hard_cv_scores.mean()
hard_cv_std = hard_cv_scores.std()
print('The Voting Classifier (hard) cross-validation mean is {:.3f}'.format(hard_cv_mean));
print('The Voting Classifier (hard) cross-validation standard deviation is {:.3f}'.format(hard_cv_std))